<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/Attn_attr_cosine_sim_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook explores the relation between the model's attributions and attentions for a given example. Historically, we found that attentions are not a feasible method of explanation whereas attributions are, but attributions are also not part of a model's traditional outputs. Therefore it may be interesting to see if we can find anything with attentions by comparing them to a feasible and plausible method of explanation. This notebook is very similar to this [notebook](https://colab.research.google.com/drive/14a1tOimrRbLlXd0rbtY-UTcizR5GDd4F) and serves to present a priminary findings of what examples are interesting by iterating through the entire dataset using much few steps when computing [Integrated Gradients](https://arxiv.org/abs/1703.01365).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import dependencies

In [ ]:
pip install transformers --quiet

     |████████████████████████████████| 4.4 MB 16.0 MB/s 
     |████████████████████████████████| 6.6 MB 53.2 MB/s 
     |████████████████████████████████| 596 kB 80.7 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 


In [ ]:
pip install captum --quiet

     |████████████████████████████████| 1.4 MB 13.1 MB/s 


In [ ]:
pip install datasets --quiet

     |████████████████████████████████| 362 kB 14.8 MB/s 
     |████████████████████████████████| 1.1 MB 81.1 MB/s 
     |████████████████████████████████| 212 kB 69.0 MB/s 
     |████████████████████████████████| 140 kB 83.6 MB/s 
     |████████████████████████████████| 127 kB 81.6 MB/s 
     |████████████████████████████████| 144 kB 67.8 MB/s 
     |████████████████████████████████| 271 kB 86.2 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
pip install rbo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import pandas as pd

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Import model

In [ ]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'danielhou13/longformer-finetuned_papers_v2'
#model_path = 'danielhou13/longformer-finetuned-news-cogs402'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/567M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

## Import dataset

Here we import the papers dataset

In [ ]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["test"]

Downloading:   0%|          | 0.00/739 [00:00<?, ?B/s]

Using custom data configuration danielhou13--cogs402dataset-144b958ac1a53abb


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-144b958ac1a53abb/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Here we import the news dataset

In [ ]:
# cogs402_ds2 = load_dataset('hyperpartisan_news_detection', 'bypublisher')['validation']
# val_size = 5000
# val_indices = np.random.randint(0, len(cogs402_ds2), val_size)
# val_ds = cogs402_ds2.select(val_indices)
# labels2 = map(int, val_ds['hyperpartisan'])
# labels2 = list(labels2)
# val_ds = val_ds.add_column("labels", labels2)

## Get Attributions

We need to create a custom forward function for use in our [Integrated Gradients](https://arxiv.org/abs/1703.01365) functions. Specifially the output we want from the forward pass of the model is the softmaxed logits, which indicate the probabilities of predicting each class for the given example.

In [ ]:
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    return output.logits

In [ ]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None, attention_mask=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask
                   )
    return torch.softmax(preds, dim = 1)

To get the attributions, we perform Integrated Gradients using the model's embeddings and pass in our custom forward function.

In [ ]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

Create functions that give us the input ids and the position ids for the text we want to examine. Furthermore, it also returns the baselines we want for integrated gradients. In this case, every token in our baseline, is a padding token.

In [ ]:
max_length = 2046
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)

    #taken from the longformer implementation
    mask = input_ids.ne(ref_token_id).int()
    incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
    position_ids = incremental_indices.long().squeeze() + ref_token_id

    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

The attributions returned has very high dimensionality and we just want a single number for every token in our example, so we sum over the last dimension and squeeze the result to get an array of shape (seq_len). You may notice that we are not normalizing the attributions here. It's okay because we will normalize it later.

In [ ]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    return attributions

For use in later functions, we want to store the attributions we find in a dictionary where the key is the example number.

In [ ]:
all_attributions = {}

On the other hand, if you have a dictionary of attributions already saved, you can import it as follows. Replace the path with a path to your own dictionary.

In [ ]:
all_attributions = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/example_attrib_dict_all.pt')

In this block of code, we iterate over the entire dataset, obtain the input_ids, position_ids, attention_mask and the baseline for integrated gradients, perform integrated gradients, and store the result in the dictionary. If you have already loaded your attributions, you can skip this step. Increase the number of steps if you desire, but it may take a very long time to run. At the end of an iteration, we also save the dictionary.

Note: the attributions will be with respect to the positive class, meaning positive attributions have more influence in the model predicting positive and negative attributions will be more influential in predicting negative.

In [ ]:
from tqdm import tqdm

# for i in tqdm(range(len(cogs402_ds))):
#   if str(i) not in all_attributions:
#     #get input ids, position ids and attention mask for integrated gradients
#     text = cogs402_ds['text'][i]
#     label = cogs402_ds['labels'][i]

#     input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
#     position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
#     attention_mask = construct_attention_mask(input_ids)

#     attributions = lig.attribute(inputs=input_ids,
#                                       baselines=ref_input_ids,
#                                       additional_forward_args=(position_ids, attention_mask),
#                                       target=1,
#                                       n_steps=50,
#                                       internal_batch_size = 2)

#     attributions_sum = summarize_attributions(attributions)

#     all_attributions[str(i)] = attributions_sum.detach().cpu().numpy()

#     torch.save(all_attributions, '/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/example_attrib_dict_all.pt')

## Get Attentions

We then get the attentions and global attentions so we can compare with the attributions.

A unique property of the longformer model is that the matrix output for the attention is not a seq_len x seq_len output. Each token can only attend to the preceeding w/2 tokens and the succeeding w/2 tokens, dictated by whatever you choose the model's attention window w to be. Another name for this is called the sliding window attention. Therefore, we need to convert sliding attention matrix to correct seq_len x seq_len matrix to remain consistent with other types of Transformer Neural Networks.

To do so, we run the following 4 functions. Our attentions will change from a tensor of shape (layer, batch, head, seq_len, x + attention_window + 1) to a tensor of shape (layer, batch, head, seq_len, seq_len). More information about the functions can be found [here](https://colab.research.google.com/drive/1Kxx26NtIlUzioRCHpsR8IbSz_DpRFxEZ).

In [ ]:
def create_head_matrix(output_attentions, global_attentions):
    new_attention_matrix = torch.zeros((output_attentions.shape[0], 
                                      output_attentions.shape[0]))
    for i in range(output_attentions.shape[0]):
        test_non_zeroes = torch.nonzero(output_attentions[i]).squeeze()
        test2 = output_attentions[i][test_non_zeroes[1:]]
        new_attention_matrix_indices = test_non_zeroes[1:]-257 + i
        new_attention_matrix[i][new_attention_matrix_indices] = test2
        new_attention_matrix[i][0] = output_attentions[i][0]
        new_attention_matrix[0] = global_attentions.squeeze()[:output_attentions.shape[0]]
    return new_attention_matrix


def attentions_all_heads(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = create_head_matrix(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_layers(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = attentions_all_heads(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

Some heads may be more important than others so we scale each attention matrix by their respective head and layer. The notebook used to get head importance is [here](https://colab.research.google.com/drive/1O4QCi8ewBp7asegKqySRflTQZ9HeH8mQ?usp=sharing). However, its possible that you might not want to scale the attentions, in which case you can ignore this section.

In [ ]:
def scale_by_importance(attention_matrix, head_importance):
  new_matrix = np.zeros_like(attention_matrix)
  for i in range(attention_matrix.shape[0]):
    head_importance_layer = head_importance[i]
    new_matrix[i] = attention_matrix[i] * np.expand_dims(head_importance_layer, axis=(1))
  return new_matrix

In [ ]:
head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/papers/pretrained/head_importance.pt")
# head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/news/head_importance.pt")

The following block of code creates a new dictionary of attention matrices. Each key corresponds to their respective example in the dataset (range 0 - dataset length) and stores a layer x head x seq_len matrix of attentions for each key

In [ ]:
all_attentions = {}

On the other hand, if you have a dictionary of attentions already saved, you can import it as follows. Replace the path with a path to your own dictionary.

In [ ]:
all_attentions = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/full_attention_matrices/papers_atten_summed_dict.pt')

In this block of code, we iterate over the entire dataset, obtain the input_ids, position_ids, attention_mask, pass the inputs into the model, obtain the output, convert the attention matrix and store the result in the dictionary. If you have already loaded your attentions, you can skip this step. Increase the number of steps if you desire, but it may take a very long time to run. At the end of every 10 iterations, we also save the dictionary. We do every 10 as saving dictionaries of attentions takes a bit of time.

In [ ]:
from tqdm import tqdm

# for i in tqdm(range(len(cogs402_ds))):
#   if str(i) not in all_attentions:
#     #get input ids, position ids and attention mask for integrated gradients
#     text = cogs402_ds['text'][i]
#     label = cogs402_ds['labels'][i]

#     input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
#     position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
#     attention_mask = construct_attention_mask(input_ids)

#     output = model(input_ids.cuda(), attention_mask=attention_mask.cuda(), labels=torch.tensor(label).cuda(), output_attentions = True)

#     batch_attn = output[-2]
#     output_attentions = torch.stack(batch_attn).cpu().squeeze()
#     global_attention = output[-1]
#     output_global_attentions = torch.stack(global_attention).cpu().squeeze()

#     converted_mat = all_layers(output_attentions, output_global_attentions).detach().cpu().numpy()
    
#     attention_matrix_summed = converted_mat.sum(axis=2)
#     all_attentions[str(i)] = attention_matrix_summed

#     if i%10 == 9:
#       torch.save(all_attentions, '/content/drive/MyDrive/cogs402longformer/results/papers/full_attention_matrices/papers_atten_summed_dict.pt')

100%|██████████| 1070/1070 [5:41:10<00:00, 19.13s/it]


### Scaling the Attention

We will then scale the summed attention by head importance. If you do not wish to scale the attentions, there is a section later in the notebook that does not perform scaling. However, we will do so here.

The following are two dictionaries of attention weights for each token (how much each token is attended to), weighted by head importance, for layer 12 and over all layers. The number of keys in the dictionary is 1070 (number of items in the validation set) and each key contains an array of shape (seq_len).

In [ ]:
# all_attentions_final = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/full_attention_matrices/example_atten_dict_12.pt')
# all_attentions_all = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/full_attention_matrices/example_atten_dict_all.pt')

Another example of importing dictionaries of attentions. These two dictionaries store the summed attentions for the layers 1-6 and 7-12 respectively. 

In [ ]:
all_attentions_lower = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/full_attention_matrices/example_atten_dict_lower.pt')
all_attentions_upper = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/full_attention_matrices/example_atten_dict_upper.pt')

This block of code iterates through the entire dataset, scales the attention matrix for each example by head importance (optional), and sums up the attention.

In this case, when taking a specific layer, you pick the layer you want (replace 11 with whatever layer you wish) and then we sum over all of the heads.

When taking a range of layers, you either want to specify a range (e.g. attention_matrix_summed[0:6]) or leave as it is to sum over all layers. Then we sum up the layers and the heads.

Lastly, we save the dictionary of summed attentions for easy access on repeat runs or other notebooks.

The two dictionaries of attentions are currently labeled _lower and _upper for the use case, but should be changed to fit the task at hand.

In [ ]:
# all_attentions_lower = {}
# all_attentions_upper = {}
# for i in tqdm(range(len(cogs402_ds))):
#   if str(i) not in all_attentions_lower and str(i) not in all_attentions_upper:

#     att_mat = all_attentions[str(i)]

#     att_mat = scale_by_importance(att_mat, head_importance)

      # how to pick a range of layers
#     att_mat_low = att_mat[0:6]
#     att_mat_up = att_mat[6:]

#     # Sum the attentions for the last layer and over all layers
#     attention_lower_layer = att_mat_low.sum(axis=1)
#     attention_lower_layer = attention_lower_layer.sum(axis=0)
#     all_attentions_lower[str(i)] = attention_lower_layer

#     attention_upper_layer = att_mat_up.sum(axis=1)
#     attention_upper_layer = attention_upper_layer.sum(axis=0)
#     all_attentions_upper[str(i)] = attention_upper_layer

#     #template for single layer
#     # attention_single_layer = att_mat[layer].sum(axis=0)
#     # all_attentions_layer[str(i)] = attention_single_layer
    
#     torch.save(all_attentions_lower, '/content/drive/MyDrive/cogs402longformer/results/papers/full_attention_matrices/example_atten_dict_lower.pt')
#     torch.save(all_attentions_upper, '/content/drive/MyDrive/cogs402longformer/results/papers/full_attention_matrices/example_atten_dict_upper.pt')

100%|██████████| 1070/1070 [03:00<00:00,  5.93it/s]


## Finding the Similarities 

By using the attention and attribution dictionaries we created, we can now do a series of comparisons for each example in our dataset.

Short helper function to normalize the attentions and attributions. 

In [ ]:
def normalize(data):
    return (data - np.min(data)) / (np. max(data) - np.min(data))

The following block of code iterates through the entire dataset, grabs the appropriate attributions and attentions from their respective dictionaries, and computes the cosine similarities, [Kendalltau](https://www.jstor.org/stable/2332226), and [Rank-biased Overlap (RBO)](https://dl.acm.org/doi/10.1145/1852102.1852106). Note that the array of attributions will be absolute valued for all cases other than the raw similarities. We absolute value the attributions because negative attributions would not necessarily mean that they have the lowest attention, rather they might have really high attention as they are more likely to help the model predict the negative class, and might be something the attentions picked up by the model. The output is a dataframe containing the computed similarities.

For the cosine similarites, we are comparing the raw un-normalized attributions and attentions, the normalized arrays, arrays with values below the median masked, arrays with values below the mean masked, and the ranks of the tokens

More information about the similarities can be found [here](https://colab.research.google.com/drive/14a1tOimrRbLlXd0rbtY-UTcizR5GDd4F#scrollTo=gB7w8fsMHFT2)

In [ ]:
from numpy.linalg import norm
import scipy.stats as stats
import rbo

In [ ]:
def get_sim_dataframe(cogs402_ds, all_attentions, all_attributions):

  dataframe = []

  for i in tqdm(range(len(cogs402_ds))):
    exam_attrib = all_attributions[str(i)]
    attention_final_layer = all_attentions[str(i)]
    
    exam_attrib = exam_attrib[:len(all_attentions[str(i)])]

    #raw sims
    cosine_raw = np.dot(exam_attrib, attention_final_layer) / (norm(exam_attrib)*norm(attention_final_layer))

    attention_final_layer2 = normalize(attention_final_layer)

    exam_attrib2 = np.abs(exam_attrib)
    exam_attrib2 = normalize(exam_attrib2)
    
    #normalized sims
    cosine = np.dot(exam_attrib2, attention_final_layer2) / (norm(exam_attrib2)*norm(attention_final_layer2))

    exam_attrib3 = np.abs(exam_attrib)
    exam_attrib3 = normalize(exam_attrib3)
    median_exam = np.percentile(exam_attrib3, 50)
    exam_attrib3[exam_attrib3 < median_exam] = 0

    attention_final_layer3 = np.copy(attention_final_layer)
    attention_final_layer3 = normalize(attention_final_layer3)
    median_12 = np.percentile(attention_final_layer3, 50)
    attention_final_layer3[attention_final_layer3 < median_12] = 0


    #sim for attributions and attentions above the median
    cosine_med = np.dot(exam_attrib3, attention_final_layer3) / (norm(exam_attrib3)*norm(attention_final_layer3))

    exam_attrib4 = np.abs(exam_attrib)
    exam_attrib4 = normalize(exam_attrib4)
    mean_exam = np.mean(exam_attrib4)
    exam_attrib4[exam_attrib4 < mean_exam] = 0

    attention_final_layer4 = np.copy(attention_final_layer)
    attention_final_layer4 = normalize(attention_final_layer4)
    mean_12 = np.mean(attention_final_layer4)
    attention_final_layer4[attention_final_layer4 < mean_12] = 0

    #sim for attributions and attentions above the mean
    cosine_mean = np.dot(exam_attrib4, attention_final_layer4) / (norm(exam_attrib4)*norm(attention_final_layer4))

    exam_attrib_rank = np.abs(exam_attrib)
    order_attrib = exam_attrib_rank.argsort()
    ranks_attrib = order_attrib.argsort()

    attention_final_layer_rank = np.copy(attention_final_layer)
    order = attention_final_layer_rank.argsort()
    ranks = order.argsort()

    #sim using the ranks of the tokens
    cosine_rank = np.dot(ranks_attrib, ranks) / (norm(ranks_attrib)*norm(ranks))

    # other similarity metrics like RBO and Kendalltau
    tau, p_value = stats.kendalltau(ranks_attrib, ranks)

    rbo_sim = rbo.RankingSimilarity(order_attrib, order).rbo()

    d = {'example': i, 'similarity normalized': cosine, 'similarity raw': cosine_raw, 'sim_norm w/ median threshold':cosine_med, 'sim_norm w/ mean threshold':cosine_mean, "sim w/ ranks":cosine_rank,
        "kendalltau": tau, "rbo":rbo_sim}
    dataframe.append(d)

  return pd.DataFrame(dataframe)

Apply the above function for our two cases, upper layers, and lower layers.

In [ ]:
df = get_sim_dataframe(cogs402_ds, all_attentions_lower, all_attributions)

100%|██████████| 1070/1070 [00:06<00:00, 152.93it/s]


In [ ]:
df2 = get_sim_dataframe(cogs402_ds, all_attentions_upper, all_attributions)

100%|██████████| 1070/1070 [00:07<00:00, 152.36it/s]


In [ ]:
df

,example,similarity normalized,similarity raw,sim_norm w/ median threshold,sim_norm w/ mean threshold,sim w/ ranks,kendalltau,rbo
0,0,0.055101,0.059613,0.052692,0.049383,0.771270,0.056225,0.502536
1,1,0.165902,-0.136923,0.150249,0.132758,0.766861,0.046023,0.509705
2,2,0.109427,-0.096491,0.103792,0.096039,0.771890,0.059434,0.509233
3,3,0.076973,0.061766,0.074296,0.065375,0.812320,0.168871,0.550735
4,4,0.151239,-0.114314,0.136400,0.121803,0.775222,0.069042,0.516904
...,...,...,...,...,...,...,...,...
1065,1065,0.090096,-0.052967,0.086513,0.076633,0.785083,0.094114,0.513477
1066,1066,0.057741,-0.033787,0.055671,0.047590,0.786249,0.094733,0.513198
1067,1067,0.074530,-0.061406,0.072185,0.064172,0.842639,0.248469,0.579818
1068,1068,0.146087,-0.094186,0.131836,0.116295,0.763927,0.039321,0.499422


Since we have converted our similarities into a dataframe, we can easily find things like the mean and the max of the similarities, which we can use to pick out an example if needed.

In [ ]:
df.max()

example                         1069.000000
similarity normalized              0.285755
similarity raw                     0.167277
sim_norm w/ median threshold       0.276364
sim_norm w/ mean threshold         0.267324
sim w/ ranks                       0.868214
kendalltau                         0.319805
rbo                                0.599629
dtype: float64

In [ ]:
df.min()

example                         0.000000
similarity normalized           0.012736
similarity raw                 -0.266211
sim_norm w/ median threshold    0.010828
sim_norm w/ mean threshold      0.003529
sim w/ ranks                    0.727784
kendalltau                     -0.059373
rbo                             0.453769
dtype: float64

In [ ]:
df2.max()

example                         1069.000000
similarity normalized              0.547884
similarity raw                     0.429935
sim_norm w/ median threshold       0.541587
sim_norm w/ mean threshold         0.528510
sim w/ ranks                       0.850276
kendalltau                         0.273140
rbo                                0.595922
dtype: float64

In [ ]:
df.mean()

example                         534.500000
similarity normalized             0.102567
similarity raw                    0.000039
sim_norm w/ median threshold      0.095146
sim_norm w/ mean threshold        0.083200
sim w/ ranks                      0.782399
kendalltau                        0.088036
rbo                               0.521297
dtype: float64

### Storing our Results

Given that this is meant to be preliminary findings about the comparison of attributions and attentions, you likely want to run through this multiple times with multiple configurations to see which configuration gives the most interesting results. As such, we want to store the results found above in a manner that we can easily access but also easy to view. Below we demonstrate how we stored the results of the means in a .csv file that we export.

We convert our results into a dictionary so we can later create a dataframe and store this information. Notably, we want the attention this example operates on in the form of text (e.g. attrib_vs_layer_12), which means **you will have to change the text depending on your context**, the mean cosine similarities, the mean cosine similarities for the ranks, the kendall tau coefficient, the RBO, as well as whether they are scaled, and/or alphanumeric tokens only.

In [ ]:
result_dataframe = []
d = {'example': "attrib_vs_lower_attn", 'mean_cosine_sim': df.mean()["similarity normalized"].round(6), "mean_cosine_sim_ranks":df.mean()["sim w/ ranks"].round(6),
        "mean_kendall_tau": df.mean()["kendalltau"].round(6), "mean_RBO":df.mean()["rbo"].round(6), "scaled":True, "alnum_only":False}
result_dataframe.append(d)


In [ ]:
df2.mean()

example                         534.500000
similarity normalized             0.180892
similarity raw                    0.019859
sim_norm w/ median threshold      0.174104
sim_norm w/ mean threshold        0.157438
sim w/ ranks                      0.782365
kendalltau                        0.089739
rbo                               0.531374
dtype: float64

We do the same, changing the example text to ...upper_attn rather than lower attention because df2 stores the values comparing attribution against upper attention layers.

In [ ]:
d2 = {'example': "attrib_vs_upper_attn", 'mean_cosine_sim': df2.mean()["similarity normalized"].round(6), "mean_cosine_sim_ranks":df2.mean()["sim w/ ranks"].round(6),
        "mean_kendall_tau": df2.mean()["kendalltau"].round(6), "mean_RBO":df2.mean()["rbo"].round(6), "scaled":True, "alnum_only":False}
result_dataframe.append(d2)

### Alphanumeric Tokens only

The following block of code iterates through the entire dataset in the same manner as the above version, grabs the appropriate attributions and attentions from their respective dictionaries, and computes the cosine similarities, kendall tau coefficients, and the RBO. The output is a dataframe containing the computed similarities. The difference is that this block of code masks the values of tokens that are non-alphanumeric (e.g. ".,][?") before calculating similarities.

More information about the similarities can be found [here](https://colab.research.google.com/drive/14a1tOimrRbLlXd0rbtY-UTcizR5GDd4F#scrollTo=gB7w8fsMHFT2)

In [ ]:
def get_sim_dataframe_alpha(cogs402_ds, all_attentions, all_attributions):

  dataframe = []

  for i in tqdm(range(len(cogs402_ds))):
    exam_attrib = all_attributions[str(i)]
    attention_final_layer = all_attentions[str(i)]
    
    exam_attrib = exam_attrib[:len(all_attentions[str(i)])]

    #input_ids
    text = cogs402_ds['text'][i]
    input_ids, _, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
    indices = input_ids[0].detach().tolist()
    all_tokens_curr = tokenizer.convert_ids_to_tokens(indices)

    exam_tokens = all_tokens_curr
    alpha_neumeric_nums = [idx for idx, element in enumerate(exam_tokens) if element.isalnum()]
    mask = np.ones(attention_final_layer.shape,dtype=bool) 
    mask[alpha_neumeric_nums] = False

    attention_final_layer[mask] = 0
    exam_attrib[mask] = 0

    #raw attributions
    cosine_raw = np.dot(exam_attrib, attention_final_layer) / (norm(exam_attrib)*norm(attention_final_layer))

    #normalized attributions
    attention_final_layer2 = normalize(attention_final_layer)

    exam_attrib2 = np.abs(exam_attrib)
    exam_attrib2 = normalize(exam_attrib2)
    
    cosine = np.dot(exam_attrib2, attention_final_layer2) / (norm(exam_attrib2)*norm(attention_final_layer2))

    #sim for attributions and attentions above the median
    exam_attrib3 = np.abs(exam_attrib)
    exam_attrib3 = normalize(exam_attrib3)
    median_exam = np.percentile(exam_attrib3, 50)
    exam_attrib3[exam_attrib3 < median_exam] = 0

    attention_final_layer3 = np.copy(attention_final_layer)
    attention_final_layer3 = normalize(attention_final_layer3)
    median_12 = np.percentile(attention_final_layer3, 50)
    attention_final_layer3[attention_final_layer3 < median_12] = 0

    cosine_med = np.dot(exam_attrib3, attention_final_layer3) / (norm(exam_attrib3)*norm(attention_final_layer3))

    #sim for attributions and attentions above the mean
    exam_attrib4 = np.abs(exam_attrib)
    exam_attrib4 = normalize(exam_attrib4)
    mean_exam = np.mean(exam_attrib4)
    exam_attrib4[exam_attrib4 < mean_exam] = 0

    attention_final_layer4 = np.copy(attention_final_layer)
    attention_final_layer4 = normalize(attention_final_layer4)
    mean_12 = np.mean(attention_final_layer4)
    attention_final_layer4[attention_final_layer4 < mean_12] = 0
    
    cosine_mean = np.dot(exam_attrib4, attention_final_layer4) / (norm(exam_attrib4)*norm(attention_final_layer4))

    #sim using the ranks of the tokens
    exam_attrib_rank = np.abs(exam_attrib)
    order_attrib = exam_attrib_rank.argsort()
    ranks_attrib = order_attrib.argsort()

    attention_final_layer_rank = np.copy(attention_final_layer)
    order = attention_final_layer_rank.argsort()
    ranks = order.argsort()

    
    cosine_rank = np.dot(ranks_attrib, ranks) / (norm(ranks_attrib)*norm(ranks))

    tau, p_value = stats.kendalltau(ranks_attrib, ranks)

    rbo_sim = rbo.RankingSimilarity(order_attrib, order).rbo()

    d = {'example': i, 'similarity normalized': cosine, 'similarity raw': cosine_raw, 'sim_norm w/ median threshold':cosine_med, 'sim_norm w/ mean threshold':cosine_mean, "sim w/ ranks":cosine_rank,
        "kendalltau": tau, "rbo":rbo_sim}
    dataframe.append(d)

  return pd.DataFrame(dataframe)

Apply the above function for our two cases, upper layers, and lower layers.

In [ ]:
df_alpha = get_sim_dataframe_alpha(cogs402_ds, all_attentions_lower, all_attributions)

100%|██████████| 1070/1070 [03:21<00:00,  5.31it/s]


In [ ]:
df_alpha2 = get_sim_dataframe_alpha(cogs402_ds, all_attentions_upper, all_attributions)

100%|██████████| 1070/1070 [03:21<00:00,  5.31it/s]


In [ ]:
df_alpha.mean()

example                         534.500000
similarity normalized             0.287514
similarity raw                   -0.005378
sim_norm w/ median threshold      0.265075
sim_norm w/ mean threshold        0.242415
sim w/ ranks                      0.875731
kendalltau                        0.371863
rbo                               0.783688
dtype: float64

Similar to the previous time we saved our results to a dictionary, but this time we change alnum_only to True because df_alpha masks all the non-alphanumeric tokens.

In [ ]:
d_alpha = {'example': "attrib_vs_lower_attn", 'mean_cosine_sim': df_alpha.mean()["similarity normalized"].round(6), "mean_cosine_sim_ranks":df_alpha.mean()["sim w/ ranks"].round(6),
        "mean_kendall_tau": df_alpha.mean()["kendalltau"].round(6), "mean_RBO":df_alpha.mean()["rbo"].round(6), "scaled":True, "alnum_only":True}
result_dataframe.append(d_alpha)

In [ ]:
df_alpha2.mean()

example                         534.500000
similarity normalized             0.417837
similarity raw                    0.005613
sim_norm w/ median threshold      0.403510
sim_norm w/ mean threshold        0.380196
sim w/ ranks                      0.877220
kendalltau                        0.376542
rbo                               0.783973
dtype: float64

The same principle for d_alpha is applied here for the upper layers.

In [ ]:
d_alpha2 = {'example': "attrib_vs_upper_attn", 'mean_cosine_sim': df_alpha2.mean()["similarity normalized"].round(6), "mean_cosine_sim_ranks":df_alpha2.mean()["sim w/ ranks"].round(6),
        "mean_kendall_tau": df_alpha2.mean()["kendalltau"].round(6), "mean_RBO":df_alpha2.mean()["rbo"].round(6), "scaled":True, "alnum_only":True}
result_dataframe.append(d_alpha2)

## Unscaled Versions

These two dictionaries store the summed attentions for the layers 1-6 and 7-12 respectively. You can change the name and the path to suit your project

In [ ]:
all_attentions_unscale_lower = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/full_attention_matrices/example_atten_dict_lower_unscale.pt')
all_attentions_unscale_upper = torch.load('/content/drive/MyDrive/cogs402longformer/results/papers/full_attention_matrices/example_atten_dict_upper_unscale.pt')

FileNotFoundError: ignored

The following block of code sums the attention in the same manner as the above [version](https://colab.research.google.com/drive/1-FZYN7yBm2jVsZds8Q4lydasdxJRe9zk#scrollTo=ZWeFQNW8MmJL&line=1&uniqifier=1), producing arrays of attention with shape (seq_len) and storing it in a dictionary.

Feel free to skip this block of code if you have already imported your attentions.

In [ ]:
# all_attentions_unscale_lower = {}
# all_attentions_unscale_upper = {}
# for i in tqdm(range(len(cogs402_ds))):
#   if str(i) not in all_attentions_unscale_lower and str(i) not in all_attentions_unscale_upper:

#     att_mat = all_attentions[str(i)]

#     att_mat_low = att_mat[0:6]
#     att_mat_up = att_mat[6:]

#     # Sum the attentions for the last layer and over all layers
#     attention_lower_layer = att_mat_low.sum(axis=1)
#     attention_lower_layer = attention_lower_layer.sum(axis=0)
#     all_attentions_unscale_lower[str(i)] = attention_lower_layer

#     attention_upper_layer = att_mat_up.sum(axis=1)
#     attention_upper_layer = attention_upper_layer.sum(axis=0)
#     all_attentions_unscale_upper[str(i)] = attention_upper_layer
    
#     torch.save(all_attentions_unscale_lower, '/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/example_atten_dict_lower_unscale.pt')
#     torch.save(all_attentions_unscale_upper, '/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/full_attention_matrices/example_atten_dict_upper_unscale.pt')

100%|██████████| 1070/1070 [02:53<00:00,  6.17it/s]


We use the same functions we create previously [above](https://colab.research.google.com/drive/1-FZYN7yBm2jVsZds8Q4lydasdxJRe9zk#scrollTo=dmKF8EBP2Nqz&line=11&uniqifier=1) to get dataframes which we can use to get the mean or the max.

In [ ]:
df_unscale = get_sim_dataframe(cogs402_ds, all_attentions_unscale_lower, all_attributions)
df_unscale2 = get_sim_dataframe(cogs402_ds, all_attentions_unscale_upper, all_attributions)

100%|██████████| 1070/1070 [00:07<00:00, 148.52it/s]


In [ ]:
df_unscale.mean()

example                         534.500000
similarity normalized             0.044425
similarity raw                   -0.004103
sim_norm w/ median threshold      0.038792
sim_norm w/ mean threshold        0.026743
sim w/ ranks                      0.759812
kendalltau                        0.028209
rbo                               0.510167
dtype: float64

We set the value of scaled to False and alnum to False given that we did not scale the attentions when getting the similarities in df_unscale nor did we mask the non-alphanumeric tokens.

In [ ]:
d_unscale = {'example': "attrib_vs_lower_attn", 'mean_cosine_sim': df_unscale.mean()["similarity normalized"].round(6), "mean_cosine_sim_ranks":df_unscale.mean()["sim w/ ranks"].round(6),
        "mean_kendall_tau": df_unscale.mean()["kendalltau"].round(6), "mean_RBO":df_unscale.mean()["rbo"].round(6), "scaled":False, "alnum_only":False}
result_dataframe.append(d_unscale)

The same principle for d_unscale is applied here, but for the other example (in our case the upper layers).

In [ ]:
df_unscale2.mean()

example                         534.500000
similarity normalized             0.057689
similarity raw                    0.006596
sim_norm w/ median threshold      0.052625
sim_norm w/ mean threshold        0.037709
sim w/ ranks                      0.783053
kendalltau                        0.092020
rbo                               0.534586
dtype: float64

In [ ]:
d_unscale2 = {'example': "attrib_vs_upper_attn", 'mean_cosine_sim': df_unscale2.mean()["similarity normalized"].round(6), "mean_cosine_sim_ranks":df_unscale2.mean()["sim w/ ranks"].round(6),
        "mean_kendall_tau": df_unscale2.mean()["kendalltau"].round(6), "mean_RBO":df_unscale2.mean()["rbo"].round(6), "scaled":False, "alnum_only":False}
result_dataframe.append(d_unscale2)

### Alphanumeric Tokens only

We use the function we created earlier [above] in order to get the dataframe with our similarities using alphanumeric inputs only.

In [ ]:
df_unscale_alpha = get_sim_dataframe_alpha(cogs402_ds, all_attentions_unscale_lower, all_attributions)

100%|██████████| 1070/1070 [03:22<00:00,  5.29it/s]


In [ ]:
df_unscale_alpha2 = get_sim_dataframe_alpha(cogs402_ds, all_attentions_unscale_upper, all_attributions)

100%|██████████| 1070/1070 [03:22<00:00,  5.28it/s]


In [ ]:
df_unscale_alpha.mean()

example                         534.500000
similarity normalized             0.158132
similarity raw                   -0.011340
sim_norm w/ median threshold      0.136436
sim_norm w/ mean threshold        0.123483
sim w/ ranks                      0.867937
kendalltau                        0.346927
rbo                               0.776589
dtype: float64

Here we are both leaving the attentions unscaled and masking all the non_alphanumeric tokens, so we appropriately set scaled to False and "alnum_only" to True.

In [ ]:
d_unscale_alpha = {'example': "attrib_vs_lower_attn", 'mean_cosine_sim': df_unscale_alpha.mean()["similarity normalized"].round(6), "mean_cosine_sim_ranks":df_unscale_alpha.mean()["sim w/ ranks"].round(6),
        "mean_kendall_tau": df_unscale_alpha.mean()["kendalltau"].round(6), "mean_RBO":df_unscale_alpha.mean()["rbo"].round(6), "scaled":False, "alnum_only":True}
result_dataframe.append(d_unscale_alpha)

The same principle for d_unscale is applied here, but for the other example (in our case the upper layers).

In [ ]:
df_unscale_alpha2.mean()

example                         534.500000
similarity normalized             0.418432
similarity raw                    0.018761
sim_norm w/ median threshold      0.394978
sim_norm w/ mean threshold        0.369731
sim w/ ranks                      0.877759
kendalltau                        0.378284
rbo                               0.784442
dtype: float64

In [ ]:
d_unscale_alpha2 = {'example': "attrib_vs_upper_attn", 'mean_cosine_sim': df_unscale_alpha2.mean()["similarity normalized"].round(6), 
                    "mean_cosine_sim_ranks":df_unscale_alpha2.mean()["sim w/ ranks"].round(6),
                    "mean_kendall_tau": df_unscale_alpha2.mean()["kendalltau"].round(6), 
                    "mean_RBO":df_unscale_alpha2.mean()["rbo"].round(6), "scaled":False, "alnum_only":True}
result_dataframe.append(d_unscale_alpha2)

## Saving the End Results

Finally, we can combine our results into a single dataframe to store our data and be able to easily access our results. We sort our examples by name and whether or not they are scaled to keep it as organized as possible.

In [ ]:
df_new = pd.DataFrame(result_dataframe)
df_new = df_new.sort_values(['example', 'scaled'],
              ascending = [True, True])
df_new

,example,mean_cosine_sim,mean_cosine_sim_ranks,mean_kendall_tau,mean_RBO,scaled,alnum_only
4,attrib_vs_lower_attn,0.044425,0.759812,0.028209,0.510167,False,False
6,attrib_vs_lower_attn,0.158132,0.867937,0.346927,0.776589,False,True
0,attrib_vs_lower_attn,0.102567,0.782399,0.088036,0.521297,True,False
2,attrib_vs_lower_attn,0.287514,0.875731,0.371863,0.783688,True,True
5,attrib_vs_upper_attn,0.057689,0.783053,0.092020,0.534586,False,False
7,attrib_vs_upper_attn,0.418432,0.877759,0.378284,0.784442,False,True
1,attrib_vs_upper_attn,0.180892,0.782365,0.089739,0.531374,True,False
3,attrib_vs_upper_attn,0.417837,0.877220,0.376542,0.783973,True,True


If you have a previous version of this dataframe, we can load the csv using pandas and add our new results onto the dataframe by using panda's concatenate function.

In [ ]:
df_previous = pd.read_csv("/content/drive/MyDrive/cogs402longformer/results/papers/attrib_attn_sim/attrib_attn_sim_means.csv")

In [ ]:
df_previous

,example,mean_cosine_sim,mean_cosine_sim_ranks,mean_kendall_tau,mean_RBO,scaled,alnum_only
0,attrib_vs_all_layer_attn,0.059124,0.776544,0.074032,0.525490,False,False
1,attrib_vs_all_layer_attn,0.235386,0.876572,0.374509,0.783674,False,True
2,attrib_vs_all_layer_attn,0.120074,0.799616,0.134198,0.542645,True,False
3,attrib_vs_all_layer_attn,0.302617,0.876614,0.374737,0.784229,True,True
4,attrib_vs_layer12_attn,0.105481,0.777507,0.077260,0.521135,False,False
5,attrib_vs_layer12_attn,0.408307,0.881266,0.390058,0.789077,False,True
6,attrib_vs_layer12_attn,0.227269,0.811874,0.168294,0.557020,True,False
7,attrib_vs_layer12_attn,0.400218,0.884370,0.400200,0.792063,True,True
8,attrib_vs_lower_attn,0.044425,0.759812,0.028209,0.510167,False,False
9,attrib_vs_lower_attn,0.158132,0.867937,0.346927,0.776589,False,True


In [ ]:
df_previous.loc[(df_previous["alnum_only"] == True)]

,example,mean_cosine_sim,mean_cosine_sim_ranks,mean_kendall_tau,mean_RBO,scaled,alnum_only
1,attrib_vs_all_layer_attn,0.235386,0.876572,0.374509,0.783674,False,True
3,attrib_vs_all_layer_attn,0.302617,0.876614,0.374737,0.784229,True,True
5,attrib_vs_layer12_attn,0.408307,0.881266,0.390058,0.789077,False,True
7,attrib_vs_layer12_attn,0.400218,0.884370,0.400200,0.792063,True,True
9,attrib_vs_lower_attn,0.158132,0.867937,0.346927,0.776589,False,True
11,attrib_vs_lower_attn,0.287514,0.875731,0.371863,0.783688,True,True
13,attrib_vs_upper_attn,0.418432,0.877759,0.378284,0.784442,False,True
15,attrib_vs_upper_attn,0.417837,0.877220,0.376542,0.783973,True,True


In [ ]:
df_previous.loc[(df_previous["alnum_only"] == False)]

,example,mean_cosine_sim,mean_cosine_sim_ranks,mean_kendall_tau,mean_RBO,scaled,alnum_only
0,attrib_vs_all_layer_attn,0.059124,0.776544,0.074032,0.525490,False,False
2,attrib_vs_all_layer_attn,0.120074,0.799616,0.134198,0.542645,True,False
4,attrib_vs_layer12_attn,0.105481,0.777507,0.077260,0.521135,False,False
6,attrib_vs_layer12_attn,0.227269,0.811874,0.168294,0.557020,True,False
8,attrib_vs_lower_attn,0.044425,0.759812,0.028209,0.510167,False,False
10,attrib_vs_lower_attn,0.102567,0.782399,0.088036,0.521297,True,False
12,attrib_vs_upper_attn,0.057689,0.783053,0.092020,0.534586,False,False
14,attrib_vs_upper_attn,0.180892,0.782365,0.089739,0.531374,True,False


When we concatenate our two dataframes, we want to make sure we don't have any duplicate rows. We consider it a duplicate if two rows have the same example, have the same value for scaled, and the same value for alnum_only. If we find a do find duplicate rows based on the above conditions, the last occuring instance of the row, which is the instance that was obtained earlier in the notebook (and not the instance that was read from file), is kept in the dataframe.

In [ ]:
df_final = pd.concat([df_previous, df_new], ignore_index=True)

In [ ]:
df_final = df_final.drop_duplicates(['example', 'scaled', 'alnum_only'], keep='last').sort_values(['example', 'scaled'],
              ascending = [True, True])

In [ ]:
df_final

,example,mean_cosine_sim,mean_cosine_sim_ranks,mean_kendall_tau,mean_RBO,scaled,alnum_only
0,attrib_vs_all_layer_attn,0.059124,0.776544,0.074032,0.525490,False,False
1,attrib_vs_all_layer_attn,0.235386,0.876572,0.374509,0.783674,False,True
2,attrib_vs_all_layer_attn,0.120074,0.799616,0.134198,0.542645,True,False
3,attrib_vs_all_layer_attn,0.302617,0.876614,0.374737,0.784229,True,True
4,attrib_vs_layer12_attn,0.105481,0.777507,0.077260,0.521135,False,False
5,attrib_vs_layer12_attn,0.408307,0.881266,0.390058,0.789077,False,True
6,attrib_vs_layer12_attn,0.227269,0.811874,0.168294,0.557020,True,False
7,attrib_vs_layer12_attn,0.400218,0.884370,0.400200,0.792063,True,True
16,attrib_vs_lower_attn,0.044425,0.759812,0.028209,0.510167,False,False
17,attrib_vs_lower_attn,0.158132,0.867937,0.346927,0.776589,False,True


Finally, we save this dataframe we made so we can either add on to this dataframe in future explorations, or take these results for other uses.

In [ ]:
df_final.to_csv("/content/drive/MyDrive/cogs402longformer/results/papers/attrib_attn_sim/attrib_attn_sim_means.csv", index=False)